In [10]:

import os
!pip install langchain
!pip install langchain_community
!pip install PYPDF2
!pip install pandas
!pip install traceback
!pip install python-dotenv
!pip install -U langchain-openai

ERROR: Could not find a version that satisfies the requirement traceback (from versions: none)
ERROR: No matching distribution found for traceback


  Using cached openai-1.38.0-py3-none-any.whl.metadata (22 kB)
  Using cached anyio-4.4.0-py3-none-any.whl.metadata (4.6 kB)
  Using cached distro-1.9.0-py3-none-any.whl.metadata (6.8 kB)
  Using cached httpx-0.27.0-py3-none-any.whl.metadata (7.2 kB)
  Using cached sniffio-1.3.1-py3-none-any.whl.metadata (3.9 kB)
  Using cached tqdm-4.66.4-py3-none-any.whl.metadata (57 kB)
     ---------------------------------------- 0.0/41.5 kB ? eta -:--:--
     ---------------------------------------- 41.5/41.5 kB 1.0 MB/s eta 0:00:00
  Using cached httpcore-1.0.5-py3-none-any.whl.metadata (20 kB)
  Using cached h11-0.14.0-py3-none-any.whl.metadata (8.2 kB)
   ---------------------------------------- 0.0/48.2 kB ? eta -:--:--
   ---------------------------------------- 48.2/48.2 kB 2.4 MB/s eta 0:00:00
Using cached openai-1.38.0-py3-none-any.whl (335 kB)
   ---------------------------------------- 0.0/798.9 kB ? eta -:--:--
   -------------------------------- ------ 675.8/798.9 kB 14.2 MB/s eta 0:0

In [11]:
from dotenv import load_dotenv
load_dotenv()

True

In [15]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
from langchain_openai import ChatOpenAI
import json
import pandas as pd
import traceback


In [16]:
from langchain.chat_models import ChatOpenAI

In [17]:
KEY=os.getenv("OPENAI_API_KEY")

In [18]:

llm=ChatOpenAI(openai_api_key=KEY,model_name="gpt-3.5-turbo", temperature=0.5)

In [19]:
llm

ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x0000019607C1CBE0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x0000019607C2B5B0>, temperature=0.5, openai_api_key='sk-qSIt8l8w73AHQZpe0io7CB554DxDGOFhPPF2wN1SQdT3BlbkFJX0zBh32ZyqNZOuw60OSLg3a9YoMbY8LtTu6OfWUg4A', openai_proxy='')

In [21]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [22]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

"""

In [23]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "response_json"],
    template=TEMPLATE
    )

In [24]:
quiz_chain=LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)

c:\Users\devan\OneDrive\Desktop\MCQ-dev\env\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


In [25]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [26]:
quiz_evaluation_prompt=PromptTemplate(input_variables=["subject", "quiz"], template=TEMPLATE)

In [27]:
review_chain=LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)

In [28]:
generate_evaluate_chain=SequentialChain(chains=[quiz_chain, review_chain], input_variables=["text", "number", "subject", "tone", "response_json"],
                                        output_variables=["quiz", "review"], verbose=True,)

In [29]:
file_path=r"C:\Users\devan\OneDrive\Desktop\MCQ-dev\data.txt"

In [30]:
with open(file_path, 'r') as file:
    TEXT = file.read()

In [31]:
print(TEXT)

Machine learning is a subset of artificial intelligence (AI) that focuses on the development of algorithms and statistical models that enable computers to perform tasks without explicit instructions. It involves the use of patterns and inference drawn from data to allow systems to learn and improve from experience. This approach contrasts with traditional programming, where specific instructions are provided for every task. Instead, machine learning models identify patterns in data and make decisions based on those patterns.

Supervised learning is one of the primary types of machine learning. In supervised learning, models are trained on labeled data, meaning the input data is paired with the correct output. The model learns to map inputs to outputs by identifying patterns in the training data. Examples of supervised learning algorithms include linear regression, logistic regression, decision trees, and support vector machines. These techniques are widely used in applications such as 

In [32]:
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [36]:
NUMBER=5 
SUBJECT="machine learning"
TONE="simple"


In [37]:
with get_openai_callback() as cb:
    response=generate_evaluate_chain(
        {
            "text": TEXT,
            "number": NUMBER,
            "subject":SUBJECT,
            "tone": TONE,
            "response_json": json.dumps(RESPONSE_JSON)
        }
        )



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:Machine learning is a subset of artificial intelligence (AI) that focuses on the development of algorithms and statistical models that enable computers to perform tasks without explicit instructions. It involves the use of patterns and inference drawn from data to allow systems to learn and improve from experience. This approach contrasts with traditional programming, where specific instructions are provided for every task. Instead, machine learning models identify patterns in data and make decisions based on those patterns.

Supervised learning is one of the primary types of machine learning. In supervised learning, models are trained on labeled data, meaning the input data is paired with the correct output. The model learns to map inputs to outputs by identifying patterns in the training data. Examples of supervised learning algorithms include linear regression, logistic regres

In [38]:
-/\\\\\\\\\\\\]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]98op';p;

Total Tokens:2948
Prompt Tokens:2114
Completion Tokens:834
Total Cost:0.0048389999999999996


In [39]:
response

{'text': "Machine learning is a subset of artificial intelligence (AI) that focuses on the development of algorithms and statistical models that enable computers to perform tasks without explicit instructions. It involves the use of patterns and inference drawn from data to allow systems to learn and improve from experience. This approach contrasts with traditional programming, where specific instructions are provided for every task. Instead, machine learning models identify patterns in data and make decisions based on those patterns.\n\nSupervised learning is one of the primary types of machine learning. In supervised learning, models are trained on labeled data, meaning the input data is paired with the correct output. The model learns to map inputs to outputs by identifying patterns in the training data. Examples of supervised learning algorithms include linear regression, logistic regression, decision trees, and support vector machines. These techniques are widely used in applicati

In [40]:
quiz=response.get("quiz")

In [41]:
quiz=json.loads(quiz)

In [42]:
quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

In [43]:
quiz_table_data


[{'MCQ': 'What is the primary focus of machine learning?',
  'Choices': 'a: Developing software applications | b: Creating algorithms and statistical models | c: Providing explicit instructions to computers | d: Building hardware components',
  'Correct': 'b'},
 {'MCQ': 'Which type of machine learning involves training models on labeled data?',
  'Choices': 'a: Supervised learning | b: Unsupervised learning | c: Reinforcement learning | d: Deep learning',
  'Correct': 'a'},
 {'MCQ': 'What is the goal of reinforcement learning?',
  'Choices': 'a: Identifying patterns in data | b: Maximizing cumulative rewards over time | c: Training models on labeled data | d: Detecting fraud in financial transactions',
  'Correct': 'b'},
 {'MCQ': 'Which type of neural network is particularly effective for image processing?',
  'Choices': 'a: Recurrent neural networks (RNNs) | b: Convolutional neural networks (CNNs) | c: Deep neural networks | d: Q-learning networks',
  'Correct': 'b'},
 {'MCQ': 'What i

In [44]:
quiz=pd.DataFrame(quiz_table_data)

In [45]:
quiz.to_csv("machinelearning.csv",index=False)

In [46]:
from datetime import datetime
datetime.now().strftime('%m_%d_%Y_%H_%M_%S')

'08_03_2024_14_16_57'